In [ ]:
import yfinance as yf
import pandas as pd

def load_and_flatten(ticker='AAPL', period='3mo', interval='1d'):
    # 1) 데이터 로드
    df = yf.download(ticker, period=period, interval=interval)

    # 2) 인덱스/컬럼 레벨 확인
    print("Index levels:", df.index.nlevels)
    print("Columns levels:", df.columns.nlevels)
    # (원한다면) 컬럼 레벨별 값도 확인
    if df.columns.nlevels > 1:
        print("Columns MultiIndex sample:", df.columns[:5])

    # 3) 멀티인덱스 풀기
    # 3-1) 행 인덱스가 MultiIndex 면 reset_index
    if isinstance(df.index, pd.MultiIndex):
        df = df.reset_index()

    # 3-2) 컬럼이 MultiIndex 면 문자열로 합쳐서 단일 레벨로
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            "_".join([str(c) for c in col if c])  # 빈 문자열 제외
            for col in df.columns.values
        ]

    return df

# # 사용 예
# df_flat = load_and_flatten('AAPL')
# df_flat.head()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import yfinance as yf
import torch

def get_stock_summary(df, ticker='AAPL'):
    # — 1) 컬럼 목록 확인 (디버깅용)
    print("▶︎ 사용 가능한 컬럼:", df.columns.tolist())

    # — 2) 'Close' 포함 컬럼 찾기
    close_cols = [c for c in df.columns if 'Close' in c]
    if not close_cols:
        raise KeyError(f"'Close' 계열 컬럼이 없습니다. 컬럼 목록: {df.columns.tolist()}")
    close_col = close_cols[0]   # e.g. 'Close' or 'Adj Close'

    # — 3) 이동평균 계산
    df['MA20'] = df[close_col].rolling(window=20).mean()
    df = df.dropna()

    # — 4) 스칼라값 꺼내기
    latest_close = df[close_col].iloc[-1]
    prev_close   = df[close_col].iloc[-2]
    latest_ma20  = df['MA20'].iloc[-1]

    # — 5) 방향 판단
    direction = "상승" if latest_close > prev_close else "하락"
    above_ma  = "상회" if latest_close > latest_ma20 else "하회"

    # — 6) 요약문 반환
    return (
        f"{ticker} 종목의 최신 {close_col}는 {latest_close:.2f}이며, "
        f"전일 대비 {direction} 중입니다. "
        f"현재 주가는 20일 이동평균선보다 {above_ma}하고 있습니다. "
        f"최근 20일 평균은 {latest_ma20:.2f}입니다."
    )



In [ ]:
from transformers import pipeline
import torch

def get_llm_recommendation(summary,
                           model_name="mistralai/Mistral-7B-Instruct-v0.1"):
    # 1) 파이프라인 초기화 (GPU 있으면 CUDA 사용)
    device = 0 if torch.cuda.is_available() else -1
    generator = pipeline(
        "text-generation",
        model=model_name,
        tokenizer=model_name,
        device=device,
        trust_remote_code=True,
    )

    # 2) Instruct-style 포맷
    prompt = (
        "### Instruction:\n"
        "너는 주식 투자 어드바이저야. 아래 요약을 바탕으로 매수 또는 매도 하나를 추천하고, 이유를 설명해줘.\n\n"
        f"{summary}\n\n"
        "### Response:"
    )

    # 3) 생성: stop sequence로 “###” 지정, full text 반환 안 함
    out = generator(
        prompt,
        max_new_tokens=64,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        eos_token_id=generator.tokenizer.eos_token_id,
        pad_token_id=generator.tokenizer.eos_token_id,
        return_full_text=False,
        # Hugging Face v5 기준. v4 파이프라인엔 stop_sequence가 없을 수도 있습니다.
        # stop_sequence=["###"]
    )
    # out는 [{'generated_text': '<모델 답변>'}] 형태
    return out[0]['generated_text'].strip()

# 예시 사용
summary = "AAPL 종목의 최신 종가는 208.37이며…"
print(get_llm_recommendation(summary))


if __name__ == "__main__":
    ticker = "AAPL"
    # 1) 데이터 로드 및 MultiIndex 해제
    df = load_and_flatten(ticker, period='3mo', interval='1d')
    # 2) 요약 생성
    summary = get_stock_summary(df, ticker)
    print("📄 주가 요약:\n", summary, "\n")
    # 3) LLM 매수/매도 추천
    recommendation = get_llm_recommendation(summary)
    print("🤖 LLM 추천:\n", recommendation)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1.
401 Client Error. (Request ID: Root=1-680b2e19-167eaf9d3c0b0b787e072976;a2e45662-0696-4d9e-9927-75c6e67679f6)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.1 is restricted. You must have access to it and be authenticated to access it. Please log in.